Walmart Desert
---
#### Key Points to note:
- THe bank data csv had to be retained in the code, as this has the city zipcodes<br/>
- Then , I had to combine the xipcode data from bank csv with our cinsus 2018 data frame<br/>
- This was then combined with the walamrt data<br/>
- Note that, when you group by zipcode , and count , pandas is displaying 'Nan' in the columns as we are trying to dispaly smaller amount of data in a bigger data set. To avoid this issue, i had to create a separate data frame which has the ccount of stores. I have also created a csv file that just holds the zipcodes and the count of stores , so that it is easy for us to combine in Postgres .<br/>

### Importing Depencdencies

In [72]:
# Dependencies
from census import Census
from config import (census_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from us import states
from scipy.stats import linregress
from matplotlib import pyplot as plt


# Census API Key
c = Census(census_key, year=2018)

## Census Data

### Data Retrieval

In [73]:
# Run Census Search to retrieve data on all zip codes (2018 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("B01003_001E", "B17001_002E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "zip code tabulation area": "Zipcode"})

# Visualize
print(len(census_pd))
census_pd.head()

33120


,Population,Poverty Count,Zipcode
0,17242.0,10772.0,00601
1,38442.0,19611.0,00602
2,48814.0,24337.0,00603
3,6437.0,4163.0,00606
4,27073.0,11724.0,00610


### Reading the zip_bank csv,as we need the zipcodes

In [74]:
# Import the original data we analyzed earlier. Use dtype="object" to match other
census_original_data= pd.read_csv(
    "Resources/zip_bank_data.csv", dtype="object", encoding="utf-8")
census_original_data

,Zipcode,Address,Median Age,Household Income,Per Capita Income,Lat,Lng,Bank Count
0,624,"Hastings, MI 49058, USA",40.8,46777,22137,42.6306916,-85.2929384,9
1,692,"Ball, LA 71405, USA",35.8,55242,23941,31.4061799,-92.396174,12
2,730,"Great Mills, MD 20634, USA",31.9,79944,35961,38.2201614,-76.4967919,9
3,757,"Williamsport, TN 38487, USA",41.6,38125,18884,35.7310368,-87.2419299,0
4,957,"Marion, ND 58466, USA",44.5,69844,36981,46.5594224,-98.3481542,1
...,...,...,...,...,...,...,...,...
695,99102,"Dysart, PA 16636, USA",47.6,42303,21236,40.594001,-78.4865602,0
696,99506,"Santa Clara, CA 95051, USA",36.1,100504,43487,37.3598283,-121.9814354,176
697,99661,"Minneapolis, MN 55433, USA",36.3,59258,27493,45.1670628,-93.3132624,64
698,99737,"Milano, TX 76556, USA",50.7,39375,21201,30.6897762,-96.8994495,0


### Combining the Census data with the zip codes from bank data

In [75]:
# Merge the two data sets along zip code
census_data_complete = pd.merge(
    census_original_data, census_pd, how="left", on=["Zipcode", "Zipcode"])

# Remove rows missing data
census_data_complete = census_data_complete.dropna()

# Visualize
census_data_complete.head()

,Zipcode,Address,Median Age,Household Income,Per Capita Income,Lat,Lng,Bank Count,Population,Poverty Count
63,10007,"Flatonia, TX 78941, USA",45.7,40304,23115,29.7574106,-97.1574784,0,7023.0,145.0
64,10303,"Nevada, IA 50201, USA",40.4,56619,28908,42.065743,-93.4599326,7,25947.0,6542.0
65,10309,"Lukachukai, AZ 86507, USA",24.1,22009,8346,36.4106866,-109.2593642,0,33531.0,3089.0
66,10553,"Lone Pine, CA 93545, USA",40.6,32473,18444,36.5131184,-118.0888578,0,10429.0,1274.0
67,10803,"Niagara, WI 54151, USA",45.7,45813,23500,45.715354,-87.9804239,2,12544.0,537.0


### Writing the DataFrame to a csv

In [76]:
census_data_complete.to_csv("Census_data.csv", index=False, header=True)

## Walmart Data

### Reading the Walmart locations data  

In [77]:
file_df=pd.read_csv("Resources/walmartstorelocation.csv", dtype="object", encoding="utf-8")
#file_df.head()
walmart_df=file_df.filter(['zip_code', 'name', 'street_address', 'city', 'country', 'latitude', 'longitude'])
walmart_renamed = walmart_df.rename(columns={'zip_code': 'Zipcode'})
walmart_renamed

,Zipcode,name,street_address,city,country,latitude,longitude
0,72032,Conway Supercenter,1155 Hwy 65 North,Conway,US,35.10866,-92.436905
1,63801,Sikeston Supercenter,1303 S Main St,Sikeston,US,36.857394,-89.586051
2,74464,Tahlequah Supercenter,2020 S Muskogee Ave,Tahlequah,US,35.888765,-94.979859
3,72653,Mountain Home Supercenter,65 Wal Mart Dr,Mountain Home,US,36.3549565,-92.3410256
4,74017,Claremore Supercenter,1500 S Lynn Riggs Blvd,Claremore,US,36.293955,-95.627125
...,...,...,...,...,...,...,...
4649,77584,Pearland Neighborhood Market,12631 Broadway Street,Pearland,US,29.5563504,-95.4157825
4650,72712,Bentonville Gas Station,1300 S Walton Blvd,Bentonville,US,36.357486,-94.214669
4651,84106,Millcreek Pickup only,3701 S Highland Dr,Millcreek,US,40.691031,-111.846529
4652,75248,Dallas Supercenter,15757 Coit Rd,Dallas,US,32.963412,-96.770294


###  Walmart Data grouping
- Grouping the dataframe by Zip code<br/>
- Counting the number ofwalmarts in every city<br/>
- Loading this in a DataFrame named 'store_count'<br/>



In [78]:
#Grouping the dataframe by city as a city will have many zip codes and
#counting the number of zipcodes as each walmart has one assigned to it
store_count=walmart_df.groupby('zip_code')['name'].nunique()
store_count.to_frame()


,name
zip_code,
00617,1
00659,1
00662,1
00674,1
00680,1
...,...
99611,1
99615,1
99654,1


### Walamrt Data summary statistics

In [79]:
walmart_df.describe()

,zip_code,name,street_address,city,country,latitude,longitude
count,4654,4654,4654,4654,4654,4654,4654
unique,4177,3130,4647,2638,2,4653,4653
top,72712,San Antonio Supercenter,100 Walmart Dr,San Antonio,US,34.765195,-86.592346
freq,10,23,4,29,4636,2,2


### Loading the store count as a separate csv, as adding the grouped count of zipcodes gave 'NaN' for count values .

In [80]:
store_count.to_csv('Walmart_store_count.csv', index=False, header=True)

### Combining the census and walmart data

In [81]:
combined_data=pd.merge(census_data_complete, walmart_renamed, on='Zipcode')
#combined_data
combined_data=combined_data.drop(['Median Age', 'Household Income', 'Per Capita Income',
                                  'Bank Count', 'Population', 'Poverty Count','Lat', 'Lng'], axis=1)
combined_data

,Zipcode,Address,name,street_address,city,country,latitude,longitude
0,12010,"Dauphin, PA 17018, USA",Amsterdam Supercenter,101 Sanford Farms Shopping Ctr,Amsterdam,US,42.964506,-74.183699
1,12804,"Slaughters, KY 42456, USA",Queensbury Supercenter,891 Route #9,Queensbury,US,43.334668,-73.681457
2,12804,"Slaughters, KY 42456, USA",Queensbury Supercenter,24 Quaker Ridge Blvd,Queensbury,US,43.319893,-73.612395
3,13637,"Mobile, AL 36619, USA",Evans Mills Supercenter,25737 Us Route #11,Evans Mills,US,44.040509,-75.844512
4,14304,"Salt Rock, WV 25559, USA",Niagara Falls Supercenter,1540 Military Rd,Niagara Falls,US,43.094308,-78.978894
...,...,...,...,...,...,...,...,...
85,92543,"Santa Isabel, 00757, Puerto Rico",Hemet Neighborhood Market,859 W Florida Ave,Hemet,US,33.745448,-116.979639
86,93637,"Glendale, CA 91207, USA",Madera Store,1977 W Cleveland Ave,Madera,US,36.975625,-120.08061
87,94539,"Racine, WV 25165, USA",Fremont Store,44009 Osgood Rd,Fremont,US,37.511302,-121.943195
88,98270,"Sand Gap, AR 72856, USA",Marysville Supercenter,8713 64th St Ne,Marysville,US,48.055649,-122.112545


### Loading the walmart data into a cleaned csv file to load into postgres SQL 

In [82]:
combined_data.to_csv('Walmart_combined_data.csv', index=False, header=True)